In [39]:
# Import libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [40]:
# Load files
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
ss = pd.read_csv('SampleSubmission.csv')

In [41]:
# Preview train
train.head()

,child_id,data_year,child_date,child_age,child_enrolment_date,child_months_enrolment,child_grant,child_years_in_programme,child_height,child_observe_attentive,...,obs_cooking_5,obs_cooking_6,obs_heating_1,obs_heating_2,obs_heating_3,obs_heating_4,obs_heating_5,obs_heating_6,obs_heating_7,target
0,ID_SYSJ2FM0D,2022.0,2022-02-03,59.000000,NaN,NaN,NaN,NaN,NaN,Sometimes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.500000
1,ID_J5BTFOZR3,2019.0,NaN,60.163933,NaN,NaN,NaN,1st year in the programme,103.000000,Sometimes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.869999
2,ID_R00SN7AUD,2022.0,2022-03-11,69.000000,NaN,NaN,NaN,NaN,108.400002,Often,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.520000
3,ID_BSSK60PAZ,2021.0,2021-10-13,53.000000,2020-01-15,20.0,No,1st year in the programme,98.099998,Almost always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.599998
4,ID_IZTY6TC4D,2021.0,2021-10-13,57.000000,2021-10-13,0.0,NaN,2nd year in programme,114.000000,Almost always,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,76.599998


In [42]:
# Preview test
test.head()

,child_id,data_year,child_date,child_age,child_enrolment_date,child_months_enrolment,child_grant,child_years_in_programme,child_height,child_observe_attentive,...,obs_cooking_4,obs_cooking_5,obs_cooking_6,obs_heating_1,obs_heating_2,obs_heating_3,obs_heating_4,obs_heating_5,obs_heating_6,obs_heating_7
0,ID_0I0999N6S,2021.0,2021-09-20,57.000000,NaN,NaN,Yes,2nd year in programme,108.0,Almost always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ID_GQ6ONJ4FP,2021.0,2021-10-21,54.000000,2021-01-10,9.0,Yes,1st year in the programme,105.0,Almost always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ID_YZ76CVRW3,2021.0,2021-05-17,57.000000,NaN,NaN,Yes,NaN,101.5,Often,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ID_BNINCRXH8,2022.0,2022-09-09,59.334702,NaN,NaN,NaN,3rd year in programme,NaN,Almost always,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,ID_1U7GDTLRI,2021.0,2021-10-12,54.000000,2021-01-15,8.0,Yes,1st year in the programme,103.5,Often,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# Preview submission file
ss.head()

,child_id,target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,ID_0I0999N6S,0,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature
1,ID_GQ6ONJ4FP,0,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature
2,ID_YZ76CVRW3,0,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature
3,ID_BNINCRXH8,0,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature
4,ID_1U7GDTLRI,0,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature,feature


In [44]:
# identify target
target_cols = []
for i in train.columns.values:
    if i not in test.columns.values:
        target_cols.append(i)
        
target = train[target_cols]
train[target_cols].head()

,target
0,51.500000
1,55.869999
2,47.520000
3,58.599998
4,76.599998


In [45]:
features = []; cat_features = []; not_features = []
for k in train.columns[1:]:
    if train[k].isnull().sum() < 6000:
        features.append(k)
        if train[k].dtype == 'O':
            cat_features.append(k)
            print('There is '+ str(len(train[k].value_counts()))+' Class in: ' +k)
    else:
        not_features.append(k)

print('----------------------------------')
print('We have '+str(len(features)) + ' features')
print('We have '+str(len(cat_features)) + ' categorical features')
print('We have '+str(len(not_features)) + ' features that have more than 6000 of missing values')

There is 254 Class in: child_date
There is 535 Class in: child_enrolment_date
There is 4 Class in: child_grant
There is 4 Class in: child_years_in_programme
There is 4 Class in: child_observe_attentive
There is 4 Class in: child_observe_concentrated
There is 4 Class in: child_observe_diligent
There is 4 Class in: child_observe_interested
There is 2 Class in: child_gender
There is 1018 Class in: child_dob
There is 3 Class in: child_stunted
There is 4 Class in: child_age_group
There is 153 Class in: id_mn_best
There is 10 Class in: prov_best
There is 50 Class in: id_dc_best
There is 50 Class in: dc_best
There is 153 Class in: mn_best
There is 5 Class in: pra_free_play
There is 4 Class in: pra_free_play_outdoor
There is 31 Class in: pra_groupings
There is 2 Class in: pra_groupings_1
There is 2 Class in: pra_groupings_2
There is 2 Class in: pra_groupings_3
There is 2 Class in: pra_groupings_4
There is 2 Class in: pra_groupings_5
There is 3 Class in: pra_engaged
There is 4 Class in: pra_age

In [46]:
train.shape

(8585, 679)

In [47]:
# get the correlation of each feature with respect to the target (emmission)
train_corr = train.corr()['target'].abs().sort_values(ascending=False)
train_corr

target                 1.000000
child_observe_total    0.444812
child_age              0.424989
child_height           0.299639
pri_fees_amount_2_3    0.293291
                         ...   
obs_lighting_6         0.003548
pri_days               0.002841
language_match         0.002638
pri_difficult_walk     0.001561
obs_cooking_4          0.000066
Name: target, Length: 160, dtype: float64

In [56]:
max(train_corr), min(train_corr)

(1.0, 6.552845715764906e-05)

In [49]:
len(train_corr)

160

In [55]:
features = train_corr[train_corr > 0].index.to_list()[1:]

print(len(features))

159


In [51]:
train = train[features]
test  = test[features]

In [52]:
train

,child_observe_total,child_age,child_height,pri_fees_amount_2_3,teacher_emotional_total,id_dc_n,id_mn_n,pri_fees_amount,pri_fees_amount_0_1,pri_fees_amount_pv,...,count_register_race_african,id_facility,pra_class_size,pra_experience,pri_time_close_minutes,count_staff_paid_maintenance,count_register_year_2020,pqa_score_relationships,count_register_gender_other,count_register_year_2014
0,4.0,59.000000,NaN,NaN,NaN,134.0,107.0,NaN,NaN,NaN,...,NaN,761.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.0,60.163933,103.000000,NaN,7.0,367.0,NaN,NaN,NaN,NaN,...,NaN,458.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7.0,69.000000,108.400002,NaN,9.0,1448.0,1448.0,0.0,NaN,0.00000,...,NaN,925.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9.0,53.000000,98.099998,NaN,12.0,629.0,76.0,350.0,NaN,373.65045,...,38.0,308.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN
4,12.0,57.000000,114.000000,100.0,12.0,315.0,71.0,100.0,100.0,106.75727,...,0.0,1749.0,12.0,1.0,0.0,NaN,2.0,4.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8580,6.0,55.000000,102.300003,170.0,10.0,139.0,65.0,170.0,300.0,181.48737,...,69.0,1371.0,23.0,20.0,0.0,1.0,0.0,4.0,0.0,0.0
8581,4.0,55.000000,102.599998,250.0,5.0,695.0,58.0,200.0,350.0,213.51454,...,33.0,1741.0,18.0,5.0,0.0,0.0,1.0,5.0,0.0,0.0
8582,12.0,56.000000,103.800003,900.0,10.0,119.0,73.0,900.0,900.0,960.81540,...,11.0,1490.0,12.0,3.0,0.0,0.0,1.0,5.0,0.0,0.0
8583,12.0,57.000000,102.400002,NaN,10.0,422.0,422.0,0.0,NaN,0.00000,...,42.0,1718.0,NaN,NaN,30.0,NaN,0.0,NaN,0.0,0.0


In [53]:
test

,child_observe_total,child_age,child_height,pri_fees_amount_2_3,teacher_emotional_total,id_dc_n,id_mn_n,pri_fees_amount,pri_fees_amount_0_1,pri_fees_amount_pv,...,count_register_race_african,id_facility,pra_class_size,pra_experience,pri_time_close_minutes,count_staff_paid_maintenance,count_register_year_2020,pqa_score_relationships,count_register_gender_other,count_register_year_2014
0,11.0,57.000000,108.0,NaN,12.0,296.0,74.0,300.0,NaN,320.271820,...,45.0,1878.0,NaN,NaN,0.0,NaN,8.0,NaN,0.0,0.0
1,11.0,54.000000,105.0,NaN,12.0,13.0,13.0,250.0,NaN,266.893200,...,40.0,268.0,NaN,NaN,30.0,NaN,0.0,NaN,0.0,0.0
2,8.0,57.000000,101.5,NaN,NaN,366.0,117.0,450.0,NaN,480.407700,...,13.0,1683.0,NaN,NaN,0.0,NaN,3.0,NaN,0.0,0.0
3,11.0,59.334702,NaN,NaN,10.0,NaN,NaN,130.0,NaN,130.000000,...,109.0,2222.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN
4,9.0,54.000000,103.5,NaN,7.0,123.0,39.0,60.0,NaN,64.054360,...,5.0,534.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,8.0,67.318275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2072.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3676,4.0,64.000000,112.0,NaN,NaN,1448.0,1448.0,550.0,NaN,587.165000,...,5.0,1437.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,0.0
3677,12.0,59.000000,108.0,50.0,NaN,190.0,13.0,50.0,50.0,53.378635,...,0.0,1518.0,12.0,8.0,0.0,0.0,0.0,5.0,0.0,0.0
3678,2.0,52.000000,106.0,200.0,5.0,695.0,58.0,200.0,200.0,213.514540,...,52.0,195.0,14.0,11.0,0.0,NaN,0.0,8.0,0.0,0.0
